# PII MASKING Guardrails  

This Guardrails configuration sets up a three-stage safety pipeline that detects and handles PII (Personally Identifiable Information) and unsafe content before and after an LLM call.

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
import nest_asyncio
nest_asyncio.apply()

In [13]:
# !pip install spacy
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\urllib3\connectionpool.py", line 464, in _make_request
    self._validate_conn(conn)
    ~~~~~~~~~~~~~~~~~~~^^^^^^
  File "C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\urllib3\connectionpool.py", line 1093, in _validate_conn
    conn.connect()
    ~~~~~~~~~~~~^^
  File "C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\urllib3\connection.py", line 741, in connect
    sock_and_verified = _ssl_wrap_socket_and_match_hostname(
        sock=sock,
    ...<14 lines>...
        assert_fingerprint=self.assert_fingerprint,
    )
  File "C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\urllib3\connection.py", line 920, in _ssl_wrap_socket_and_match_hostname
    ssl_sock = ssl_wrap_socket(
        sock=sock,
    ...<8 lines>...
        tls_in_tls=tls_in_tls,
    )
  File "C:\Users\vyanktesh.l\Documents\FDE\genai_env\Lib\site-packages\urllib3\util\ss

In [12]:
import spacy

spacy.load('en_core_web_sm')

import asyncio
from contextlib import suppress

from rich.console import Console
from rich.panel import Panel

from guardrails import GuardrailsAsyncOpenAI, GuardrailTripwireTriggered

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

### Guardrail config

## 📌 What Each Section Does

1️⃣ pre_flight Guardrails — PII Masking Before Model Use

- Detects PII before sending the prompt to the model

- Does not block requests containing PII (block: False) → Instead, PII will be masked (e.g., john@example.com → [EMAIL_ADDRESS])

- Detects encoded PII (Base64, hex, obfuscated formats)

- Ensures sensitive data never reaches the LLM in raw form

Goal: Protect user privacy at the entry point

2️⃣ input Guardrails — Harmful Prompt Filtering

 - Uses Moderation to block prompts containing:  "hate" content "violence"

 - Prevents unsafe or abusive queries from being processed

 - Runs before the LLM is called

Goal: Stop malicious prompts early

3️⃣ output Guardrails — PII Blocking After Model Response

- Detects if the model’s response contains PII

- Here, block: True → If the model generates phone numbers, emails, etc., the response is blocked

- Ensures the system never leaks sensitive info back to users

Goal: Prevent data exposure



In [5]:

# Define pipeline configuration with PII masking
PIPELINE_CONFIG = {
    "version": 1,
    "pre_flight": {
        "version": 1,
        "guardrails": [
            {
                "name": "Contains PII",
                "config": {
                    "entities": [
                        "EMAIL_ADDRESS",
                        "PHONE_NUMBER",
                        "US_SSN",
                        "CREDIT_CARD",
                        "CVV",
                        "BIC_SWIFT",
                    ],
                    "block": False,  # Default - won't block, just mask
                    "detect_encoded_pii": True,
                },
            }
        ],
        "config": {"concurrency": 5, "suppress_tripwire": False},
    },
    "input": {
        "version": 1,
        "guardrails": [{"name": "Moderation", "config": {"categories": ["hate", "violence"]}}],
        "config": {"concurrency": 5, "suppress_tripwire": False},
    },
    "output": {
        "version": 1,
        "guardrails": [
            {
                "name": "Contains PII",
                "config": {
                    "entities": [
                        "EMAIL_ADDRESS",
                        "PHONE_NUMBER",
                        "US_SSN",
                        "CREDIT_CARD",
                    ],
                    "block": True,  # Will block output if PII is detected
                },
            }
        ],
        "config": {"concurrency": 5, "suppress_tripwire": False},
    },
}


### Process user input using GuardrailsClient with automatic PII masking
process_input() does the following: 
- Sends the user input to the Guardrails-enabled LLM only after guardrails validate it
- Ensuring PII is masked before calling the model.
- Displays the assistant’s response, highlights any PII masking done in pre-flight or output stages
- Updates the conversation history only when guardrails pass, preventing unsafe text from being stored.

In [6]:


console = Console()

async def process_input(
    guardrails_client: GuardrailsAsyncOpenAI,
    user_input: str,
    messages: list[dict],
) -> None:
    """Process user input using GuardrailsClient with automatic PII masking.

    Args:
        guardrails_client: GuardrailsClient instance with PII masking configuration.
        user_input: User's input text.
        messages: Conversation history (modified in place after guardrails pass).
    """
    try:
        # Pass user input inline WITHOUT mutating messages first
        # Only add to messages AFTER guardrails pass and LLM call succeeds
        response = await guardrails_client.chat.completions.create(
            messages=messages + [{"role": "user", "content": user_input}],
            model="gpt-4",
        )

        # Show the LLM response (already masked if PII was detected)
        content = response.llm_response.choices[0].message.content
        console.print(f"\n[bold blue]Assistant output:[/bold blue] {content}\n")

        # Show PII masking information if detected in pre-flight
        if response.guardrail_results.preflight:
            for result in response.guardrail_results.preflight:
                if result.info.get("guardrail_name") == "Contains PII" and result.info.get("pii_detected", False):
                    detected_entities = result.info.get("detected_entities", {})
                    masked_text = result.info.get("checked_text", user_input)

                    # Show what text was actually sent to the LLM
                    console.print(
                        Panel(
                            f"PII detected and masked before sending to LLM:\n"
                            f"Original: {user_input}\n"
                            f"Sent to LLM: {masked_text}\n"
                            f"Entities found: {', '.join(detected_entities.keys())}",
                            title="PII Masking Applied",
                            border_style="yellow",
                        )
                    )
        # Show if PII was detected in output
        if response.guardrail_results.output:
            for result in response.guardrail_results.output:
                if result.info.get("guardrail_name") == "Contains PII" and result.info.get("pii_detected", False):
                    detected_entities = result.info.get("detected_entities", {})
                    console.print(
                        Panel(
                            f"Warning: PII detected in LLM output (Was not masked. Set block=True to block the output):\n"
                            f"Entities found: {', '.join(detected_entities.keys())}",
                            title="PII in Output",
                            border_style="yellow",
                        )
                    )

        # Guardrails passed - now safe to add to conversation history
        messages.append({"role": "user", "content": user_input})
        messages.append({"role": "assistant", "content": content})

    except GuardrailTripwireTriggered as exc:
        stage_name = exc.guardrail_result.info.get("stage_name", "unknown")
        guardrail_name = exc.guardrail_result.info.get("guardrail_name", "unknown")
        console.print(f"[bold red]Guardrail '{guardrail_name}' triggered in stage '{stage_name}'![/bold red]")
        console.print(Panel(str(exc.guardrail_result), title="Guardrail Result", border_style="red"))
        # Guardrail blocked - user message NOT added to history
        raise



Main function :

- Initializes a GuardrailsAsyncOpenAI client using the configured PII masking rules
- sets up an initial system prompt for the conversation.
- Enters an interactive input loop where each user message is processed through process_input(), ensuring guardrails validate and mask sensitive data before adding it to history.

- Gracefully handles interruptions and guardrail-triggered violations, allowing safe, continuous interaction until the user types exit.

In [7]:

async def main() -> None:
    """Main async input loop for user interaction."""
    # Initialize GuardrailsAsyncOpenAI with PII masking configuration
    guardrails_client = GuardrailsAsyncOpenAI(config=PIPELINE_CONFIG)

    messages: list[dict] = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Comply with the user's request.",
        }
    ]

    with suppress(KeyboardInterrupt, asyncio.CancelledError):
        while True:
            try:
                user_input = input("\nEnter a message: ").strip()
                if user_input.lower() == "exit":
                    break

                await process_input(guardrails_client, user_input, messages)

            except EOFError:
                break
            except GuardrailTripwireTriggered:
                # Already handled in process_input
                continue
            except Exception as e:
                console.print(f"\nError: {e}")
                continue


if __name__ == "__main__":
    asyncio.run(main())
    console.print("\nExiting the program.")


Enter a message:  My email id is suchitra@xhy.com


Assistant output: Sorry, but I can't assist with that.

╭────────────────────────────────────────────── PII Masking Applied ──────────────────────────────────────────────╮
│ PII detected and masked before sending to LLM:                                                                  │
│ Original: My email id is suchitra@xhy.com                                                                       │
│ Sent to LLM: My email id is <EMAIL_ADDRESS>                                                                     │
│ Entities found: EMAIL_ADDRESS                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


Enter a message:  exit


Exiting the program.